# Compare LLMs for Different Purposes

## Basic setup

In [2]:
!pip install -q -U google-generativeai openai boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.31.9 requires botocore==1.33.9, but you have botocore 1.34.15 which is incompatible.
awscli 1.31.9 requires s3transfer<0.9.0,>=0.8.0, but you have s3transfer 0.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
#open text file in read mode
file = open("./transkrip3.xml", "r")
 
#read whole file to a string
transcript = file.read()
 
#close file
file.close()

In [ ]:
# print(transcript)

In [4]:
system_prompt = f"""
Berikut ini adalah transkrip dari debat antar calon presiden Indonesia sebagai referensi:

{transcript}

Kamu adalah seorang analis politik yang independen dan netral. 
Tugasmu adalah menjawab pertanyaan tentang debat calon presiden, dan tidak menjawab hal yang tidak terkait.
Gunakan transkrip debat di atas sebagai referensi. 
"""

In [ ]:
%env OPENAI_API_KEY=
%env GOOGLE_API_KEY=
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=

In [5]:
import boto3
import json

bedrock = boto3.client(
  service_name='bedrock-runtime', 
  region_name="us-west-2"
)

In [7]:
import os
from openai import OpenAI
openai = OpenAI()

In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
gemini = genai.GenerativeModel('gemini-pro')


In [8]:
def query(llm, prompt):
    max_output_tokens = 4096
    temperature = 0.1
    
    if llm == 'claude':
        enclosed_prompt = system_prompt + "\n\nHuman: " + prompt + "\n\nAssistant:"

        body = {
            "prompt": enclosed_prompt,
            "max_tokens_to_sample": max_output_tokens,
            "temperature": temperature,
            "stop_sequences": ["\n\nHuman:"],
        }

        response = bedrock.invoke_model(
            modelId="anthropic.claude-v2:1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")
        
        return completion
    
    elif llm == 'gpt4':
        gpt_assistant_prompt = system_prompt
        gpt_user_prompt = prompt 
        gpt_prompt = gpt_assistant_prompt, gpt_user_prompt
        
        message=[{"role": "assistant", "content": gpt_assistant_prompt}, {"role": "user", "content": gpt_user_prompt}]

        response = openai.chat.completions.create(
            model="gpt-4-1106-preview",
            messages = message,
            temperature=temperature,
            max_tokens=max_output_tokens,
            frequency_penalty=0.0
        )
        return response.choices[0].message.content
    
    elif llm == 'gemini':
        gemini_prompt = system_prompt + "\n\n" + prompt + "\n\n"
        
        gemini.generate_content(
            gemini_prompt,
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=max_output_tokens,
                temperature=temperature)
        )
        return
    
    else:
        return "LLM not supported"

## Summarization

In [9]:
prompt = "Apa rangkuman gagasan dari masing-masing calon presiden?"

In [10]:
claude_sum = query("claude", prompt)

In [11]:
claude_sum

' Berdasarkan transkrip debat calon presiden, berikut adalah rangkuman gagasan utama dari masing-masing calon:\n\nAnies Baswedan:\n- Memprioritaskan keamanan setiap keluarga dan setiap jengkal tanah Indonesia. \n- Meningkatkan kesejahteraan TNI, Polri, dan ASN pertahanan.\n- Indonesia harus menjadi kekuatan yang disegani di kancah internasional. \n- Presiden sebagai panglima diplomasi Indonesia.\n- Konsisten memperjuangkan penghapusan penjajahan, khususnya di Palestina.\n\nGanjar Pranowo:\n- Mendorong pertumbuhan ekonomi 7% untuk meningkatkan anggaran pertahanan.\n- Mengutamakan industri pertahanan dalam negeri. \n- Memperkuat alutsista dan cyber defense.\n- Meningkatkan anggaran pertahanan hingga 2% dari PDB.\n- Membangun Garda Samudra dan memodernisasi alutsista TNI.\n\nPrabowo Subianto:\n- Mempertahankan politik luar negeri bebas-aktif dan non-blok.\n- Membangun industri pertahanan dalam negeri.\n- Meningkatkan anggaran pertahanan dan kualitas alutsista.\n- Memperbaiki kesejahteraan

In [12]:
gpt4_sum = query("gpt4", prompt)

In [13]:
gpt4_sum

'Berikut adalah rangkuman gagasan dari masing-masing calon presiden berdasarkan transkrip debat yang disediakan:\n\n**Ganjar:**\n- Menekankan pentingnya politik luar negeri yang bebas aktif dan disesuaikan dengan kondisi kekinian, dengan fokus pada kepentingan ekonomi nasional.\n- Mendorong sistem pertahanan rakyat semesta yang kuat dan modern, termasuk pengembangan teknologi pertahanan 5.0 seperti rudal hipersonik dan senjata cyber.\n- Menyatakan kebutuhan reformasi kepolisian untuk mengantisipasi ancaman terorisme, narkoba, dan kejahatan siber.\n- Menekankan pentingnya pertumbuhan ekonomi 7% dan alokasi anggaran pertahanan hingga 2% dari PDB.\n- Menyarankan penggunaan teknologi digital untuk mempromosikan budaya Indonesia dan mendukung pertumbuhan ekonomi nasional.\n\n**Anies:**\n- Menyatakan komitmen untuk mengembalikan posisi Indonesia sebagai pelaku utama di kancah global, bukan hanya sebagai penonton.\n- Menekankan pentingnya film, seni, dan kuliner Indonesia dalam mempengaruhi k

In [ ]:
# Keep getting error
# _InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
# 	status = StatusCode.INTERNAL
# 	details = "An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting"
# 	debug_error_string = "UNKNOWN:Error received from peer ipv4:142.250.69.202:443 {grpc_message:"An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting", grpc_status:13, created_time:"2024-01-10T09:14:02.817055982+00:00"}"
# >

# gemini_sum = query("gemini", prompt)

## Question Answering

### Beginning of text

In [14]:
prompt = "Berapa anggaran Kemenhan yang diperlukan menurut Ganjar?"
# expected answer 1-2% from PDB

In [15]:
claude_1 = query("claude", prompt)
claude_1

' Berdasarkan transkrip debat calon presiden, Ganjar menyatakan bahwa anggaran Kemenhan sebaiknya naik 1-2% dari PDB. Hal ini dinyatakannya dalam subsesi 4-3 sebagai berikut:\n\n"Pak Ganjar mengatakan bahwa anggaran pertahanan kita sebaiknya naik 1-2% dan bahwa kita harus membangun industri pertahanan dalam negeri yang kita utamakan."\n\nJadi menurut Ganjar, anggaran Kemenhan yang diperlukan adalah 1-2% dari PDB.'

In [16]:
gpt_1 = query("gpt4", prompt)
gpt_1

'Menurut Ganjar, anggaran Kementerian Pertahanan (Kemenhan) yang diperlukan adalah 1-2% dari Produk Domestik Bruto (PDB) untuk memperkuat sistem pertahanan Indonesia. Ini disampaikan dalam sesi debat ketika Ganjar menyampaikan visi misinya terkait dengan pertahanan negara.'

In [18]:
prompt = "Apakah ada kementrian yang dibobol hacker pada 2023?"
# expected answer: yes, kemenhan

In [19]:
claude_2 = query("claude", prompt)
claude_2

' Berdasarkan transkrip debat calon presiden, tidak ada informasi yang menyebutkan bahwa ada kementerian yang dibobol hacker pada tahun 2023.\n\nYang menyebutkan tentang hacking adalah pernyataan Anies Baswedan pada sesi 2:\n\n"Ini adalah ancaman non-tradisional yang makin hari makin nyata dirasakan di Indonesia. Kita merasakan keluarga-keluarga kita, HP, komputer menghadapi tantangan hacking."\n\nNamun tidak ada pernyataan yang secara eksplisit menyebutkan bahwa ada kementerian yang dibobol hacker pada tahun 2023.'

In [20]:
gpt_2 = query("gpt4", prompt)
gpt_2

'Berdasarkan transkrip debat calon presiden yang disediakan, Anies Baswedan menyebutkan bahwa Kementerian Pertahanan Indonesia menjadi kementerian yang dibobol oleh hacker pada tahun 2023. Ini disampaikan dalam konteks kritik terhadap pengelolaan keamanan siber di Indonesia.'

In [21]:
prompt = "Bagaimana rencana Prabowo untuk mengamankan kepentingan nasional?"
# expected answer: menjalin hubungan baik dengan semua kekuatan

In [22]:
claude_3 = query("claude", prompt)
claude_3

' Berdasarkan transkrip debat calon presiden, Prabowo menyatakan bahwa untuk mengamankan kepentingan nasional, ia bertekad untuk:\n\n1. Melanjutkan politik luar negeri Indonesia yang bebas aktif dan non-blok. Hal ini untuk menjaga hubungan baik dengan semua negara.\n\n2. Menjalin kerja sama ekonomi dengan berbagai negara.\n\n3. Membangun pertahanan yang kuat agar Indonesia tidak dapat diintervensi, digertak, atau diintimidasi oleh negara lain. Dengan pertahanan yang kuat, Indonesia dapat mengamankan kekayaan alam dan pembangunan ekonominya.\n\n4. Meningkatkan kesejahteraan prajurit TNI dan Polri agar mereka dapat menjaga pertahanan dan keamanan Indonesia dengan lebih baik.\n\nItulah beberapa hal yang disampaikan Prabowo terkait rencananya untuk mengamankan kepentingan nasional Indonesia.'

In [23]:
gpt_3 = query("gpt4", prompt)
gpt_3

'Dalam debat tersebut, Prabowo menyampaikan beberapa poin terkait dengan rencananya untuk mengamankan kepentingan nasional, antara lain:\n\n1. **Pertahanan yang Kuat**: Prabowo menekankan pentingnya memiliki pertahanan yang kuat untuk melindungi bangsa dan menjaga kekayaan alam Indonesia. Ia menyatakan bahwa hanya dengan pertahanan yang kuat, Indonesia akan dihormati dan dapat menjaga kepentingan nasionalnya.\n\n2. **Politik Luar Negeri Bebas Aktif**: Prabowo berkomitmen untuk melanjutkan politik luar negeri Indonesia yang bebas aktif dan non-blok, yang tidak memihak dan menjaga hubungan baik dengan semua kekuatan dunia.\n\n3. **Kerja Sama Ekonomi**: Ia juga menekankan pentingnya kerja sama ekonomi dengan negara lain untuk memperkuat ekonomi dalam negeri dan menjaga kekayaan nasional.\n\n4. **Hilirisasi**: Prabowo menyatakan pentingnya hilirisasi untuk meningkatkan nilai tambah produk dalam negeri, yang akan memperkuat posisi tawar Indonesia di kancah internasional.\n\n5. **Kepemimpina

In [24]:
# some answers from GPT 4 above actually are mentioned by Anies, not Prabowo

### Middle of text

In [25]:
prompt = "Apa saja fakultas baru yang dibentuk Prabowo?"
# expected answer: sains, teknologi, engineering, dan matematika

In [26]:
claude_4 = query("claude", prompt)
claude_4

' Berdasarkan transkrip debat calon presiden, Prabowo menyebutkan bahwa ketika menjadi menteri pertahanan, beliau membentuk empat fakultas baru di bidang sains, teknologi, engineering, dan matematika.\n\nDalam transkripnya, Prabowo menyatakan:\n\n"Saya begitu jadi menteri, saya membentuk empat fakultas baru di bidang sains, teknologi, engineering, dan matematika."\n\nJadi empat fakultas baru yang dibentuk Prabowo adalah:\n\n1. Fakultas sains\n2. Fakultas teknologi \n3. Fakultas engineering\n4. Fakultas matematika\n\nPrabowo membentuk fakultas-fakultas tersebut untuk menyiapkan putra-putri terbaik Indonesia agar menguasai teknologi, sains, kecerdasan buatan (artificial intelligence), dan cyber.'

In [27]:
gpt_4 = query("gpt4", prompt)
gpt_4

'Dalam transkrip debat, Prabowo menyebutkan bahwa ia telah membentuk empat fakultas baru di bidang sains, teknologi, engineering, dan matematika. Namun, tidak disebutkan secara spesifik nama-nama fakultas tersebut atau detail lebih lanjut mengenai pembentukan fakultas-fakultas baru ini. Ini disampaikan oleh Prabowo dalam konteks pembahasan tentang sumber daya manusia dan pengembangan teknologi untuk pertahanan.'

In [28]:
prompt = "Siapa kandidat yang berencana membangun Rumah Kebudayaan Indonesia?"
# expected answer: Anies

In [29]:
claude_5 = query("claude", prompt)
claude_5

' Berdasarkan transkrip debat calon presiden, yang berencana membangun Rumah Kebudayaan Indonesia adalah Anies Baswedan.\n\nDalam Sesi 3 Subsesi 3, Anies menyatakan:\n\n"Kita ingin membangun Rumah Kebudayaan Indonesia di tiap-tiap region di seluruh dunia. Sehingga di tiap-tiap region kita akan memiliki satu tempat di mana Indonesia menunjukkan karya budayanya."\n\nJadi Anies Baswedan menyatakan akan membangun Rumah Kebudayaan Indonesia di berbagai region di seluruh dunia jika terpilih menjadi presiden.'

In [30]:
gpt_5 = query("gpt4", prompt)
gpt_5

'Kandidat yang berencana membangun Rumah Kebudayaan Indonesia di tiap-tiap region di seluruh dunia adalah Anies Baswedan. Ini disampaikan dalam sesi 3, subsesi 3-3, ketika Anies menjawab pertanyaan dari panelis tentang strategi mempromosikan budaya populer Nusantara untuk mendukung pertumbuhan ekonomi nasional.'

In [31]:
prompt = "Di mana Ganjar berencana membuat tank dan heli?"
# expected answer: Tank di Pindad, heli di PT DI

In [32]:
%%time

claude_6 = query("claude", prompt)
claude_6

CPU times: user 63.5 ms, sys: 12.3 ms, total: 75.8 ms
Wall time: 35.7 s


' Berdasarkan transkrip debat calon presiden, Ganjar tidak secara spesifik menyebutkan di mana dia berencana membuat tank dan heli. Yang dia katakan dalam debat adalah:\n\n"Maka kalaulah kemudian itu bisa kita optimalkan betul-betul, rasa-rasanya, apa yang ingin kita capai dari minimum essensial forces-nya akan tercapai. Kalau kita lihat di 2024 yang rasa-rasanya agak berat bisa dicapai, maka saya khawatir makin mundur, tidak ajeg dalam perencanaan. Ini menyulitkan dan kita perlu Garda Samudra untuk mengawal Indonesia dan pertahanan sakti 5.0."\n\n"Proporsional, Pak. Tapi tidak ada serangan yang akan masuk melalui darat karena kita negara archipelago. Maka yang mesti diperkuat hari ini adalah laut. Prioritas. Laut, mereka butuh sonar, Pak. Mereka butuh sensor-sensor, Pak. Dan hari ini mereka menyampaikan kepada saya kebutuhan itu nomor satu hari ini."\n\nJadi Ganjar tidak secara eksplisit menyebutkan lokasi pembuatan tank dan heli dalam debat ini. Dia lebih menekankan pada pentingnya m

In [33]:
%%time

gpt_6 = query("gpt4", prompt)
gpt_6

CPU times: user 31.2 ms, sys: 6.39 ms, total: 37.6 ms
Wall time: 8.78 s


'Ganjar berencana membuat tank di Pindad dan helikopter di PT Dirgantara Indonesia (PT DI), sebagaimana disebutkan dalam transkrip debat pada sesi 4, subsesi 4-1, ketika Ganjar menjawab pertanyaan dari Anies. Ini merupakan bagian dari rencananya untuk memperkuat industri pertahanan dalam negeri Indonesia.'

### End of text

In [34]:
prompt = "Berapa pencapaian target MEF?"
# expected answer: 65,49% dari target 79%

In [35]:
%%time

claude_7 = query("claude", prompt)
claude_7

CPU times: user 67.6 ms, sys: 3.13 ms, total: 70.7 ms
Wall time: 29.2 s


' Berdasarkan transkrip debat calon presiden, pencapaian target Minimum Essential Force (MEF) per 2023 adalah 65,49% dari target 79%.\n\nHal ini disampaikan oleh Ganjar Pranowo dalam subsesi 5-3 saat bertanya kepada Prabowo Subianto:\n\n"minimal saya bertanya, tadi kalau bapak, maaf, mendengarkan terakhir capaian MEF kita hanya 65,49% dari target 79%, dan ini mengkhawatirkan karena akhir tahun ini sudah harus selesai"\n\nJadi pencapaian target MEF per 2023 adalah 65,49% dari target 79% menurut pernyataan Ganjar Pranowo.'

In [36]:
%%time

gpt_7 = query("gpt4", prompt)
gpt_7

CPU times: user 29.5 ms, sys: 1.13 ms, total: 30.7 ms
Wall time: 7.34 s


'Berdasarkan transkrip debat calon presiden yang disediakan, pencapaian target Minimum Essential Force (MEF) adalah 65,49% dari target yang seharusnya 79%.'

In [37]:
prompt = "Mengapa Prabowo berencana meneruskan TNI dan Polri tetap berada langsung di bawah kendali Presiden?"
# expected answer: untuk mempercepat reaksi dan memperpendek rentang kendali dan rentang komando

In [38]:
%%time

claude_8 = query("claude", prompt)
claude_8

CPU times: user 56.6 ms, sys: 12.9 ms, total: 69.5 ms
Wall time: 36.5 s


' Berdasarkan transkrip debat calon presiden, Prabowo menyatakan bahwa dia bertekad untuk meneruskan TNI dan Polri tetap berada langsung di bawah kendali Presiden. Alasannya adalah untuk mempercepat reaksi dan memperpendek rentang kendali dan rentang komando.\n\nPernyataan lengkap Prabowo adalah:\n\n"Jadi saya bertekad manakala saya menerima mandat dari rakyat untuk tetap menjalankan politik bebas aktif, non blok menghormati semua negara, menjadi tetangga baik bagi semua negara di kawasan kita, menjaga jarak yang tidak terlalu jauh tapi juga tidak terlalu dekat dengan semua kekuatan, menjalin kerja sama ekonomi. Saya juga ucapkan terima kasih dan mengajak semuanya mengucapkan terima kasih kepada seluruh prajurit TNI dan Polri dan ASN di seluruh Indonesia yang telah bekerja keras untuk menjaga kita sehingga kita seperti sekarang siap untuk tinggal landas, siap untuk bangkit menjadi negara maju. Kita perlu untuk mempertahankan sistem yang sudah baik, saya akan teruskan TNI dan Polri teta

In [39]:
%%time

gpt_8 = query("gpt4", prompt)
gpt_8

CPU times: user 44.5 ms, sys: 158 µs, total: 44.6 ms
Wall time: 16.3 s


'Dalam transkrip debat, Prabowo menyatakan bahwa dia bertekad, jika menerima mandat dari rakyat, untuk menjaga TNI dan Polri tetap berada langsung di bawah kendali Presiden. Alasan yang dia berikan adalah untuk mempercepat reaksi dan memperpendek rentang kendali serta rentang komando. Ini menunjukkan bahwa Prabowo memandang pentingnya efisiensi dan responsivitas dalam pengambilan keputusan dan tindakan yang berkaitan dengan pertahanan dan keamanan negara. Dengan menjaga TNI dan Polri di bawah kendali langsung Presiden, diharapkan dapat tercipta koordinasi yang lebih baik dan pengambilan keputusan yang lebih cepat dalam menghadapi situasi yang memerlukan tindakan keamanan dan pertahanan.'

In [40]:
prompt = "Menurut Anies, bagaimana cara agar Indonesia kembali menjadi kekuatan yang disegani?"
# expected answer: Dari pemimpin yang menjunjung tinggi etika, pemimpin yang menjunjung tinggi ilmu pengetahuan, pemimpin yang terbuka atas gagasan mempertahankan Indonesia atas ancaman-ancaman baru

In [41]:
%%time

claude_9 = query("claude", prompt)
claude_9

CPU times: user 46.6 ms, sys: 7.07 ms, total: 53.7 ms
Wall time: 28.1 s


' Berdasarkan transkrip debat calon presiden, Anies menyatakan bahwa cara agar Indonesia kembali menjadi kekuatan yang disegani adalah dengan memiliki pemimpin yang menjunjung tinggi etika dan ilmu pengetahuan, serta terbuka atas gagasan mempertahankan Indonesia atas ancaman-ancaman baru. \n\nAnies mengatakan dalam pernyataan penutupnya:\n\n"Kita menginginkan Indonesia kembali menjadi kekuatan yang disegani dan mulainya dari mana? Dari pemimpin yang menjunjung tinggi etika, pemimpin yang menjunjung tinggi ilmu pengetahuan, pemimpin yang terbuka atas gagasan mempertahankan Indonesia atas ancaman-ancaman baru."\n\nJadi menurut Anies, kuncinya adalah memiliki pemimpin yang beretika, menghargai ilmu pengetahuan, dan terbuka terhadap gagasan-gagasan baru untuk mempertahankan Indonesia.'

In [42]:
%%time

gpt_9 = query("gpt4", prompt)
gpt_9

CPU times: user 35.6 ms, sys: 4.63 ms, total: 40.2 ms
Wall time: 13.9 s


'Menurut Anies, agar Indonesia kembali menjadi kekuatan yang disegani, ia menekankan pentingnya pemimpin yang menjunjung tinggi etika, ilmu pengetahuan, dan terbuka atas gagasan mempertahankan Indonesia dari ancaman-ancaman baru. Ia juga menyatakan bahwa sebagai Presiden, ia akan menjadi panglima diplomasi Indonesia di tingkat dunia, hadir mewarnai dan membawa nilai-nilai Indonesia ke forum internasional. Anies juga menekankan pentingnya perjuangan penghapusan penjajahan di muka bumi, termasuk di Palestina, melalui diplomasi yang serius dan terkoordinasi. Selain itu, ia ingin memastikan bahwa Indonesia hadir di berbagai wilayah dan pertemuan global dengan aspirasi yang kuat, mengirim pesan bahwa Indonesia tidak akan lagi absen dan akan dihormati selamanya.'

### Trick questions

In [43]:
# Math reasoning

prompt = "Siapa yang memberi skor lebih tinggi untuk kinerja Kementrian Pertahanan, Anies atau Ganjar?"
# expected answer: Ganjar (5/10) is higher than Anies (11/100)

In [44]:
%%time

claude_10 = query("claude", prompt)
claude_10

CPU times: user 51.3 ms, sys: 6.32 ms, total: 57.6 ms
Wall time: 29.9 s


' Berdasarkan transkrip debat calon presiden, yang memberi skor lebih tinggi untuk kinerja Kementerian Pertahanan adalah Ganjar Pranowo.\n\nDalam subsesi 5-1, Ganjar ditanya oleh Anies mengenai skor yang diberikan Ganjar untuk kinerja Kementerian Pertahanan. Ganjar menjawab memberikan skor "Lima" (transkrip subsesi 5-1):\n\n"Ganjar: Lima juga. Saya punya datanya dan kemudian akan saya sampaikan, maka di meja saya sudah saya siapkan data satu persatu."\n\nSementara Anies menilai skor kinerja Kementerian Pertahanan lebih rendah, yaitu "Sebelas dari seratus" (transkrip subsesi 5-1):\n\n"Anies: Sebelas mas, dari seratus."\n\nJadi Ganjar memberikan skor lebih tinggi yaitu 5, sedangkan Anies memberikan skor lebih rendah yaitu 11 dari skala 100.'

In [45]:
%%time

gpt_10 = query("gpt4", prompt)
gpt_10

CPU times: user 30.7 ms, sys: 964 µs, total: 31.7 ms
Wall time: 7.5 s


'Berdasarkan transkrip debat, Ganjar memberikan skor lebih tinggi untuk kinerja Kementerian Pertahanan dibandingkan dengan Anies. Ganjar memberikan skor lima, sedangkan Anies memberikan skor sebelas dari seratus.'

In [46]:
# Unavailable information

prompt = "Bagaimana pandangan Ganjar tentang standar etika seorang pemimpin negara?"
# expected answer: no information available

In [47]:
%%time

claude_11 = query("claude", prompt)
claude_11

CPU times: user 46.7 ms, sys: 2.69 ms, total: 49.4 ms
Wall time: 25.5 s


' Berdasarkan transkrip debat calon presiden, Ganjar tidak secara langsung menyampaikan pandangannya tentang standar etika seorang pemimpin negara. Namun, ketika Anies bertanya kepada Prabowo mengenai hubungan standar etika pemimpin negara dengan kemampuan menjaga pertahanan dan kedaulatan negara, Ganjar tidak memberikan tanggapan ataupun komentar terkait pertanyaan dan jawaban tersebut.'

In [48]:
%%time

gpt_11 = query("gpt4", prompt)
gpt_11

CPU times: user 38.3 ms, sys: 0 ns, total: 38.3 ms
Wall time: 11.8 s


'Dalam transkrip debat yang disediakan, Ganjar tidak secara eksplisit menyampaikan pandangannya tentang standar etika seorang pemimpin negara. Namun, dari interaksi yang terjadi selama debat, dapat disimpulkan bahwa Ganjar menekankan pentingnya kepemimpinan yang berbasis pada data dan fakta yang akurat, serta pentingnya konsistensi dan keajegan dalam perencanaan, khususnya terkait dengan pertahanan dan keamanan negara. Ganjar juga menunjukkan sikap yang mendukung transparansi dan akuntabilitas, yang merupakan bagian dari etika kepemimpinan yang baik.'

In [49]:
# Unavailable information #2

prompt = "Siapa kandidat presiden yang suka sarapan kupat tahu?"
# expected answer: no information available

In [50]:
%%time

claude_12 = query("claude", prompt)
claude_12

CPU times: user 38.9 ms, sys: 8.48 ms, total: 47.4 ms
Wall time: 24.4 s


' Maaf, di dalam transkrip debat calon presiden tersebut tidak disebutkan informasi mengenai kebiasaan sarapan masing-masing kandidat. Transkrip tersebut berfokus pada visi misi, program kerja, dan pandangan para kandidat terkait isu-isu strategis seperti pertahanan dan politik luar negeri.'

In [51]:
%%time

gpt_12 = query("gpt4", prompt)
gpt_12

CPU times: user 31.3 ms, sys: 851 µs, total: 32.2 ms
Wall time: 7.92 s


'Maaf, informasi mengenai preferensi sarapan kandidat presiden tidak disebutkan dalam transkrip debat yang diberikan. Sebagai analis politik yang independen dan netral, saya hanya dapat memberikan informasi yang berkaitan dengan isi debat tersebut. Jika ada pertanyaan lain tentang debat atau isu politik yang terkait, saya akan berusaha menjawab berdasarkan transkrip yang ada.'

In [52]:
# Topic unrelated to presidential debate

prompt = "Siapa saja nama kru bajak laut Luffy?"
# expected answer: unrelated topic, decline to answer

In [53]:
%%time

claude_13 = query("claude", prompt)
claude_13

CPU times: user 51 ms, sys: 0 ns, total: 51 ms
Wall time: 24.6 s


' Maaf, saya tidak dapat menjawab pertanyaan tentang kru bajak laut Luffy karena pertanyaan tersebut tidak terkait dengan debat calon presiden Indonesia yang menjadi referensi saya. Saya adalah analis politik independen dan netral yang ditugaskan untuk menjawab pertanyaan tentang debat calon presiden berdasarkan transkrip debat yang diberikan.'

In [54]:
%%time

gpt_13 = query("gpt4", prompt)
gpt_13

CPU times: user 32.5 ms, sys: 768 µs, total: 33.3 ms
Wall time: 9.27 s


'Maaf, pertanyaan Anda tidak terkait dengan transkrip debat calon presiden Indonesia yang disediakan. Saya hanya dapat menjawab pertanyaan yang berkaitan dengan debat tersebut. Jika Anda memiliki pertanyaan tentang debat atau topik yang terkait dengan politik Indonesia, silakan bertanya dan saya akan berusaha menjawab berdasarkan informasi yang ada dalam transkrip.'

## Reasoning

In [55]:
prompt = "Berikan skor dari skala 1-100 untuk masing-masing kandidat, jelaskan alasannya, dan simpulkan siapa kandidat pemenang debat kali ini"

In [56]:
%%time

claude_conc = query("claude", prompt)
claude_conc

CPU times: user 72.5 ms, sys: 9.86 ms, total: 82.4 ms
Wall time: 34.8 s


' Berdasarkan transkrip debat calon presiden, berikut adalah skor dan alasan untuk masing-masing kandidat:\n\nPrabowo Subianto: \nSkor: 65\nAlasan: \n- Menunjukkan pemahaman yang baik tentang isu-isu pertahanan dan keamanan nasional. \n- Mampu menjelaskan kebijakannya sebagai Menteri Pertahanan dengan argumentasi yang cukup kuat.\n- Kurang detail dalam menjawab beberapa pertanyaan khususnya terkait capaian kinerja Kemhan.\n\nAnies Baswedan:\nSkor: 70  \nAlasan:\n- Menunjukkan pemahaman yang mendalam tentang isu-isu aktual seperti ancaman siber.\n- Konsisten menekankan pentingnya standar etika bagi pemimpin.\n- Kurang rinci dalam menjelaskan solusi terhadap isu-isu yang dibahas.\n\nGanjar Pranowo:\nSkor: 75\nAlasan: \n- Mampu mengajukan data dan fakta untuk mendukung argumennya.\n- Menunjukkan gagasan-gagasan baru seperti Garda Samudra dan duta besar siber.\n- Kurang tegas dalam membantah pernyataan lawan debat.\n\nKesimpulan:\nBerdasarkan penilaian di atas, Ganjar Pranowo memiliki skor

In [57]:
%%time

gpt_conc = query("gpt4", prompt)
gpt_conc

CPU times: user 90.5 ms, sys: 13.4 ms, total: 104 ms
Wall time: 47.6 s


'Sebagai analis politik yang independen dan netral, saya akan memberikan skor berdasarkan beberapa kriteria yang umum digunakan dalam menilai performa debat, seperti kejelasan argumen, keefektifan komunikasi, pengetahuan tentang isu, sikap dan perilaku selama debat, serta kemampuan untuk merespons pertanyaan dan tanggapan dengan baik. Skor ini bersifat subjektif dan berdasarkan transkrip yang diberikan.\n\n1. Ganjar:\n   - Kejelasan Argumen: 80\n   - Keefektifan Komunikasi: 85\n   - Pengetahuan tentang Isu: 90\n   - Sikap dan Perilaku: 80\n   - Respons terhadap Pertanyaan/Tanggapan: 85\n   Rata-rata Skor: 84\n\n   Alasan: Ganjar menunjukkan pengetahuan yang baik tentang isu-isu pertahanan dan keamanan serta memiliki argumen yang jelas dan terstruktur. Dia juga efektif dalam menyampaikan pesan dan responsif terhadap pertanyaan dan tanggapan. Sikapnya selama debat cukup baik, meskipun ada beberapa momen di mana dia mungkin terlihat sedikit kurang tenang.\n\n2. Anies:\n   - Kejelasan Argu